In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)

import shared_utils

In [2]:
## parameters cell
itp_id = 300

In [3]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [4]:
analysis_date = shared_utils.rt_dates.DATES['aug2024']

In [5]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_name.str.contains('Torr'))

In [6]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
30,City of Santa Monica,300,recJHFDLpGRMIFgnL,07 - Los Angeles,True,2024-08-14,efbbd5293be71f7a5de0cf82b59febe1,speedmap_segs_available


In [7]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [8]:
# speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')` #  non-aggregated

In [9]:
speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_speedmap_segments_{analysis_date}.parquet') #  aggregated

In [10]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry'],
      dtype='object')

In [11]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [12]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
415656,efbbd5293be71f7a5de0cf82b59febe1,dd518afc867853fe8a713c0d27a5b69d,26973,1421__1422,VIA MARINA SB & PANAY FS__VIA MARINA SB & MARQ...,1421-1422-1,offpeak,27.99,2,25.26,30.73,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"LINESTRING (-118.45918 33.97903, -118.45918 33..."
415657,efbbd5293be71f7a5de0cf82b59febe1,dd518afc867853fe8a713c0d27a5b69d,26973,1421__1422,VIA MARINA SB & PANAY FS__VIA MARINA SB & MARQ...,1421-1422-1,peak,19.87,2,19.66,20.07,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"LINESTRING (-118.45918 33.97903, -118.45918 33..."
415658,efbbd5293be71f7a5de0cf82b59febe1,dd518afc867853fe8a713c0d27a5b69d,26973,1421__1422,VIA MARINA SB & PANAY FS__VIA MARINA SB & MARQ...,1421-1422-1,all_day,21.82,4,19.93,27.08,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"LINESTRING (-118.45918 33.97903, -118.45918 33..."


In [13]:
speedmap_segs.time_period.unique()

array(['offpeak', 'peak', 'all_day'], dtype=object)

In [14]:
peak_test = speedmap_segs >> filter(_.time_period == 'peak')

In [15]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [16]:
time_periods

[(1, 'Owl'),
 (2, 'Owl'),
 (3, 'Owl'),
 (4, 'Early AM'),
 (5, 'Early AM'),
 (6, 'Early AM'),
 (7, 'AM Peak'),
 (8, 'AM Peak'),
 (9, 'AM Peak'),
 (10, 'Midday'),
 (11, 'Midday'),
 (12, 'Midday'),
 (13, 'Midday'),
 (14, 'Midday'),
 (15, 'PM Peak'),
 (16, 'PM Peak'),
 (17, 'PM Peak'),
 (18, 'PM Peak'),
 (19, 'PM Peak'),
 (20, 'Evening'),
 (21, 'Evening'),
 (22, 'Evening'),
 (23, 'Evening'),
 (24, 'Evening')]

In [17]:
total_peak = len([x[1] for x in time_periods if 'Peak' in x[1]])

In [18]:
total_peak

8

In [19]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / total_peak

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [20]:
peak_test = prep_gdf(peak_test)

In [21]:
cmap = shared_utils.rt_utils.ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.SPEEDMAP_LEGEND_URL

##  adding identifiers for map display

In [22]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,trips_per_hour
415663,efbbd5293be71f7a5de0cf82b59febe1,dd518afc867853fe8a713c0d27a5b69d,26973,1423__1568,VIA DOLCE NB & DELL ALLEY FS__WASHINGTON EB & ...,1423-1568-1,peak,8.36,1,8.36,8.36,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"POLYGON ((142005.205 -447132.952, 142004.394 -...",0.125
415762,efbbd5293be71f7a5de0cf82b59febe1,dd518afc867853fe8a713c0d27a5b69d,26973,568__161,MONTANA EB & 24TH FS__MONTANA EB & 26TH FS,568-161-1,peak,18.64,1,18.64,18.64,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"POLYGON ((139872.655 -440637.194, 139874.076 -...",0.125


In [23]:
gpd.read_file('300_AM_Peak_speeds.geojson') >> head(3)

,id,stop_id,stop_name,shape_id,stop_sequence,route_id,route_short_name,direction_id,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour,miles_from_last,time_formatted,organization_name,geometry
0,306,1129,MAIN NB & OLYMPIC FS,26731,2.0,3709,5,0.0,3.4,3.4,3.4,1.0,0.3,0.1,1:45,City of Santa Monica,"POLYGON ((-118.49075 34.01082, -118.49099 34.0..."
1,307,292,2ND NB & BROADWAY NS,26731,3.0,3709,5,0.0,9.2,9.2,9.2,1.0,0.3,0.3,1:57,City of Santa Monica,"POLYGON ((-118.49316 34.01241, -118.49383 34.0..."
2,308,104,SANTA MONICA EB & 3RD FS,26731,4.0,3709,5,0.0,13.0,13.0,13.0,1.0,0.3,0.2,0:55,City of Santa Monica,"POLYGON ((-118.49673 34.01480, -118.49673 34.0..."


##  to-do

* where does new n_trips come from? should be schedule-based for speedmaps applications...
* split am/pm peak
* https://github.com/cal-itp/data-analyses/issues/1218 upstream add route identifiers

In [24]:
peak_test.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry', 'trips_per_hour'],
      dtype='object')

## export map

In [25]:
speedmap_state = shared_utils.rt_utils.set_state_export(peak_test, filename='testnew6', map_type='speedmap',
                                      color_col='p20_mph', cmap=cmap, legend_url=url,
                                              cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/testnew6.geojson.gz


In [26]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-08-14',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/testnew6.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.01668600960056, -118.46977869668166),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMDgtMTQiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdG5ldzYuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzQuMDE2Njg2MDA5NjAwNTYsIC0xMTguNDY5Nzc4Njk2NjgxNjZdLCAiem9vbSI6IDEzLCAibGVnZW5kX3V